# LoTSS DR1
## PyBDSF - Optical associations over the HETDEX field

***

This notebook outputs a table with the association between the raw PyBDSF catalogue and the final optical catalogue. 
It includes:
* The [Original catalogues](#Loading-the-original-catalogues) 
* The [Cleaned catalogues](#Cleaning-the-original-catalogues)
* The [Ouput catalogue](#Output-table)

The output catalogue is a table which consists of:
* `pybdsf_name` : PyBDSF raw catalogue ` Source_Name `
* `association_name` : optical v1.2 catalogue `Source_Name`
* `flag`:  
    * [Deblended sources](#Deblended-sources) (`flag=4`)
    * [Multi-component PyBDSF sources](#Multi-component-PyBDSF-sources) (`flag=8`)
    * [Single sources](#Single-sources) (`flag=1`)
    * [Artifacts](#Artifacts) (`flag=16` and `flag=32`)
    * or a combination of categories: 
       * [deblended + multi-component PyBDSF sources](#Sources-that-were-both-deblended-and-associated-to-another-source) (`flag=12`)
       * [single sources that went through deblending but were not deblended](#Sources-that-do-not-seem-to-be-deblended) (`flag=3`) 

** Libraries ** 

In [1]:
import pandas as pd
import numpy as np
from astropy.table import Table, join
import unittest

***
## Catalogues
***

### Loading the original catalogues

In [2]:
# Creating a function to read the fits catalogues
def read_fits(file):
    'converts a fits table to pandas format'
    cat = Table.read(file)
    return cat.to_pandas()

In [3]:
# Raw PyBDSF catalogue
pybdsf = read_fits('../data/LOFAR_HBA_T1_DR1_catalog_v0.9.srl.fixed.fits')
# V1.2 LoTSS DR1 catalogues
optical = read_fits('../data/v1.2/LOFAR_HBA_T1_DR1_merge_ID_optical_f_v1.2.fits')
components = read_fits('../data/v1.2/LOFAR_HBA_T1_DR1_merge_ID_v1.2.comp.fits')
# Artifacts catalogue
artifacts = read_fits('../data/LOFAR_HBA_T1_DR1_merge_ID_v1.1.art.fits')

### Cleaning the original catalogues

####  Renaming Sources

Renaming a source that was picked on 2 different mosaics (it has the same `Source_Name`) and which was classified as an artifact in one mosaic and as a source in other. The source is renamed with the original name followed by 'B'. This is done on both **Artifacts catalogue** and **PyBDSF raw catalogue** (the source appears twice on the PyBDSF catalogue).

In [4]:
# Taking the name of the source (duplicated entry) from the pybdsf catalogue
pybdsf_duplicated = np.array(pybdsf[pybdsf.duplicated('Source_Name')]['Source_Name'])[0]
pybdsf_duplicated

'ILTJ132633.10+484745.7'

##### Renaming on the Artifact catalogue 

In [5]:
# Creating a copy for the new artifact catalogue
artifacts_cleaned = artifacts.copy()
# Replacing the original name by 'nameB'
clean_art = artifacts_cleaned['Source_Name'].replace({pybdsf_duplicated:pybdsf_duplicated+'B'})
# Updating the new catalogue
artifacts_cleaned.update(clean_art)

##### Renaming on the Pybdsf catalogue

In [6]:
# Creating a copy for the new pybdsf catalogue
pybdsf_cleaned = pybdsf.copy()
# Replacing the original name of the artifact on pybdsf catalogue
clean_pybdsf = pybdsf[pybdsf.duplicated('Source_Name', keep = 'last')].\
        replace({pybdsf_duplicated:pybdsf_duplicated+'B'})
# Updating the new catalogue
pybdsf_cleaned.update(clean_pybdsf)

#### Eliminating duplicated entries

`Source_Name` entries are duplicated in LoTSS DR1 ** Components catalogue ** (9 equal `Souce_Name` components). These are entries where `Source_Name = Component_Name` (which is equal to pybdsf name).

In [7]:
# Eliminating these from the components catalogue (and just keeping the first entry)
clean_comp = components.drop_duplicates('Component_Name', keep = 'first')
len(clean_comp)

324615

#### Eliminating a component on the components catalogue that is an artifact

In [8]:
# This is an artifact from the artifact catalogue that was not removed from the components catalogue
component_artifact = artifacts_cleaned[artifacts_cleaned['Source_Name'].
                                       isin(clean_comp['Component_Name'])]
component_artifact['Source_Name']

728    ILTJ115320.73+552641.4
Name: Source_Name, dtype: object

In [9]:
components_cleaned = clean_comp[~clean_comp['Component_Name'].
                                isin(component_artifact['Source_Name'])]
len(components_cleaned)

324614

***
## Diagnosis output table outline
***

### Creating a diagnosis output table outline

In [10]:
col_names =  ['pybdsf_name', 'association_name', 'flag']
output_df  = pd.DataFrame(columns = col_names)
output_df

,pybdsf_name,association_name,flag


### Creating empty lists to store the different PyBDSF association groups 

In [11]:
pybdsf_deblended_df = []
pybdsf_multi_df = []
pybdsf_singles_df = []
pybdsf_artifacts_df = []

### Creating a unit test

This test is used, throughout the notebook, to check if the different association groups are being correctly appended to the output table.

In [12]:
#Creating a function to be tested
def len_output_df(artifacts, singles, deblends, multiples):
    return len(artifacts)+ len(singles)+len(deblends)+len(multiples)

In [13]:
# Create a test case
class Test_output(unittest.TestCase):
    # Create the unit test
    def test_len_output_df(self):
        # Test if length of output table is the expected one
        self.assertEqual(len(output_df), len_output_df(pybdsf_artifacts_df,\
                                                       pybdsf_deblended_df,\
                                                       pybdsf_multi_df,\
                                                       pybdsf_singles_df))

In [14]:
# Run the unit test
unittest.main(argv=['ignored', '-v'], exit=False)

test_len_output_df (__main__.Test_output) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


***
## Deblended sources
***


Deblended sources are distint sources that were originally associated by PyBDSF as only one source, i.e. different PyBDSF components were associated into the same `Source_Name` in the **Compoments Catalogue**.

### Notes about deblended sources

According to Williams et al. 2018 these sources were sent to a deblend workflow (`ID_flag = 41`) or selected for deblending on LGZ (`ID_flag = 42`).

In [15]:
# on the optical catalogue and on the components catalogue
len(optical[(optical['ID_flag'] == 41) | (optical['ID_flag'] == 42)]),\
len(components_cleaned[(components_cleaned['ID_flag'] == 41) | (components_cleaned['ID_flag'] == 42)].\
    groupby('Source_Name'))

(2435, 2435)

However note that some of them were not actually deblended (and some may be part of a bigger source):

In [16]:
# Taking the number of sources with flags 41 or 42 that do not present any 'Deblended_from' source
len(optical[((optical['ID_flag'] == 41) | (optical['ID_flag'] == 42)) &\
            (optical['Deblended_from'] == '')]),\
len(components_cleaned[((components_cleaned['ID_flag'] == 41) | (components_cleaned['ID_flag'] == 42)) &\
               (components_cleaned['Deblended_from'] == '')].groupby('Source_Name'))

(641, 641)

In [17]:
# Taking the number of sources with 'Deblended_from' information
len(optical[((optical['ID_flag'] == 41) | (optical['ID_flag'] == 42)) &\
            (optical['Deblended_from'] != '')]),\
len(components_cleaned[((components_cleaned['ID_flag'] == 41) | (components_cleaned['ID_flag'] == 42)) &\
               (components_cleaned['Deblended_from'] != '')].groupby('Source_Name'))

(1794, 1794)

Note also that some of the `Deblended_from` sources in the optical catalogue are not original PyBDSF sources but sources names that were created from LGZ positions.

In [18]:
# Number of sources where that happens
len(optical[((optical['ID_flag'] == 41) | (optical['ID_flag'] == 42)) &\
            (optical['Deblended_from'] != '')]
    [~optical[((optical['ID_flag'] == 41) | (optical['ID_flag'] == 42)) &\
            (optical['Deblended_from'] != '')]['Deblended_from'].\
     isin(pybdsf_cleaned['Source_Name'])])

70

### Selecting the deblended sources

Deblended sources are selected from the **components catalogue** (which links directly to the PyBDSF catalogue through `Deblended_from` column).

#### Having a look at the number of sources

In [19]:
# Total number of PyBDSF sources that were deblended
len(components_cleaned[components_cleaned['Deblended_from'] != ''].groupby('Deblended_from'))

935

In [20]:
# Total number of components that make these PyBDSF
components_deblended = components_cleaned[components_cleaned['Deblended_from'] != '']
len(components_deblended)

2446

In [21]:
# Total number of sources after deblending
len(components_deblended.groupby('Source_Name'))

1794

#### Taking the PyBDSF Source Names

We have to search each PyBDSF in `components_deblended` and take the correct names:

In [22]:
deblended_pybdsf_names = components_deblended.drop_duplicates('Deblended_from')['Deblended_from']
len(deblended_pybdsf_names)

935

This has to be done this way because 1 PyBDSF source can have been deblended into 2 (or more) sources directly; or deblended and associated to other source. For that reason after droping duplicated association names (`Source_Name` below), we miss some of the original PyBDSF names:

In [23]:
len((components_deblended.drop_duplicates('Source_Name')).groupby('Deblended_from'))

908

#### Creating the deblended table

In [24]:
pybdsf_deblended_df = pd.DataFrame(columns = col_names)
for i in range(0,len(deblended_pybdsf_names)):
    corresp = components_deblended[components_deblended['Deblended_from'] == deblended_pybdsf_names.
                                   iloc[i]][['Deblended_from','Source_Name']]
    pybdsf_deblended = pd.DataFrame(
        {'pybdsf_name':deblended_pybdsf_names.iloc[i],
        'flag':4,
        'association_name': corresp.drop_duplicates()['Source_Name']})
    pybdsf_deblended_df = pybdsf_deblended_df.append(pybdsf_deblended)

This table is no longer a 1-1 relationship:

In [25]:
pybdsf_deblended_df.describe()

,association_name,flag,pybdsf_name
count,1832,1832,1832
unique,1794,1,935
top,ILTJ141420.38+462620.3,4,ILTJ112051.72+474517.1
freq,4,1832,3


#### Recover information about the Component Names

In [26]:
# We have lost information about 'Component_Name' for these deblended sources
# since a 'Deblended_from' PyBDSF source can be made up of several components
# This information can be recovered later if needed:
len(components_deblended[components_deblended['Deblended_from'].\
                                          isin(pybdsf_deblended_df['pybdsf_name'])])

2446

### Sources that were both deblended and associated to another source

Some of the deblended sources were also associated into a bigger source. These are now selected and flagged accordingly (i.e. flagged as deblended + multi-component sources).

#### Selecting the sources

In [27]:
# Taking the deblended sources that appear more than once on components deblended
cond_deblended_multi = pd.DataFrame([components_deblended['Source_Name'].value_counts() > 1][0])
len(cond_deblended_multi)

1794

In [28]:
# Taking the names for these sources (i.e. optical source names in the components catalogue)
deblended_multi_names = pd.DataFrame(cond_deblended_multi[cond_deblended_multi['Source_Name']].index.values)
len(deblended_multi_names)

209

In [29]:
# Selecting all the sources that correspond to these names
deblended_multi = pybdsf_deblended_df[pybdsf_deblended_df['association_name'].\
                                      isin(deblended_multi_names[0])]
len(deblended_multi)

247

#### Updating the deblended table

In [30]:
pybdsf_deblended_multi = deblended_multi.copy()
pybdsf_deblended_multi['flag'] = 12

In [31]:
pybdsf_deblended_df.update(pybdsf_deblended_multi)

In [32]:
len(pybdsf_deblended_df[pybdsf_deblended_df['flag'] == 12])

247

In [33]:
print pybdsf_deblended_df['flag'].value_counts()
pybdsf_deblended_df.describe()

4.0     1585
12.0     247
Name: flag, dtype: int64


,association_name,flag,pybdsf_name
count,1832,1832.0,1832
unique,1794,2.0,935
top,ILTJ141420.38+462620.3,4.0,ILTJ112051.72+474517.1
freq,4,1585.0,3


### Sources that do not seem to be deblended 

Sources where the component got the same name the pybdsf (= deblended from)

Some of these sources seem that were deblended - because they have the deblended from, but pybdsf name = component name = source name, just appear once on the component cat (eg below)

In [34]:
# Taking the sources that kept the same name after deblending (component name and source name)
deblended_same = components_cleaned[
    (components_cleaned['Deblended_from'] == components_cleaned['Source_Name']) & 
    (components_cleaned['Deblended_from'] == components_cleaned['Component_Name'])]
len(deblended_same)

11

In [35]:
# Looking at these ones on the output table 
deblended_same_output = pybdsf_deblended_df[pybdsf_deblended_df['association_name'].
                    isin(deblended_same['Source_Name'])]
len(deblended_same_output)

12

In [36]:
deblended_same_output['flag'].value_counts()

4.0     10
12.0     2
Name: flag, dtype: int64

In [37]:
# The one(s) with flag 'deblended' + 'multi' (flag 12)
# might have been associated and then deblended
# However the ones with flag 'deblended' (flag 4) do not seem to have been deblended

# Selecting the sources that were flagged as deblends (flag 4)
deblended_singles = deblended_same_output[(deblended_same_output['flag'] == 4) & 
                    (deblended_same_output['association_name'] ==
                     deblended_same_output['pybdsf_name']) ]
len(deblended_singles)

10

In [38]:
# Creating a table for these singles with flag 3 
deblended_singles_df = deblended_singles.copy()
deblended_singles_df['flag'] = 3
deblended_singles_df.head()

,association_name,flag,pybdsf_name
25451,ILTJ110953.70+482721.1,3,ILTJ110953.70+482721.1
34760,ILTJ111744.40+484605.9,3,ILTJ111744.40+484605.9
68204,ILTJ114330.16+474155.0,3,ILTJ114330.16+474155.0
154213,ILTJ124615.48+503126.5,3,ILTJ124615.48+503126.5
154253,ILTJ124617.13+503126.5,3,ILTJ124617.13+503126.5


In [39]:
pybdsf_deblended_df.update(deblended_singles_df)
# Flag counts
pybdsf_deblended_df['flag'].value_counts()

4.0     1575
12.0     247
3.0       10
Name: flag, dtype: int64

In [40]:
pybdsf_deblended_df[(pybdsf_deblended_df['flag'] == 4) | (pybdsf_deblended_df['flag'] == 12) ].describe()

,association_name,flag,pybdsf_name
count,1822,1822.0,1822
unique,1784,2.0,925
top,ILTJ141420.38+462620.3,4.0,ILTJ112051.72+474517.1
freq,4,1575.0,3


In [41]:
#deblended_singles_df = pybdsf_deblended_df.update(deblended_singles_df)

In [42]:
# Updating the deblend table 
#pybdsf_deblended_df = pybdsf_deblended_df.combine_first(deblended_singles_df)

### Updating the output table

In [43]:
output_df = output_df.append(pybdsf_deblended_df,ignore_index=True) 
unittest.main(argv=['ignored', '-v'], exit=False)

test_len_output_df (__main__.Test_output) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.007s

OK


In [44]:
output_df.describe()

,association_name,flag,pybdsf_name
count,1832,1832.0,1832
unique,1794,3.0,935
top,ILTJ141420.38+462620.3,4.0,ILTJ112051.72+474517.1
freq,4,1575.0,3



### Notes
The pybdsf sources that were deblended into several components and after associated have `S_Code`= M on the optical catalogue! 

In [45]:
deblended_optical = optical[optical['Source_Name'].isin(pybdsf_deblended_df['association_name'])]
len(deblended_optical)

1794

Taking the ones with the M code

In [46]:
deblended_associated = deblended_optical[deblended_optical['S_Code']=='M']
len(deblended_associated)

209

In [47]:
deblended_associated_df = pd.DataFrame({
    'pybdsf_name':deblended_associated['Deblended_from'],
    'flag':'M',
    'association_name': deblended_associated['Source_Name']})

Confirming that they are the same:

In [48]:
len(deblended_associated[deblended_associated['Source_Name'].
                         isin(pybdsf_deblended_df['association_name'])]),\
len(pybdsf_deblended_df[pybdsf_deblended_df['association_name'].
                        isin(deblended_associated['Source_Name'])]),\
len(pybdsf_deblended_df[pybdsf_deblended_df['association_name'].
                        isin(deblended_associated['Source_Name'])].
                        groupby('association_name'))

(209, 247, 209)

***
## Multi-component PyBDSF sources
***

Sources that were associated (but not have been deblended)

IMPORTANT: Note that S_code (M) that comes from pybdsf catalogue (and is on components catalogue and optical) does not mean that the sources are an association from several components, while the M code on optical catalogue that comes from deblended sources means that.

### Selecting the multi PyBDSF component sources

In [49]:
# Creating a condition to select the sources that are made up of more than one component
cond_multi = pd.DataFrame([components_cleaned['Source_Name'].value_counts() > 1][0])

In [50]:
# It returns the names of the sources 
cond_multi.head(2)

,Source_Name
ILTJ140313.38+542018.9,True
ILTJ121903.03+471642.8,True


In [51]:
# Taking the source names that meet the condition (names correspond to the indexes)
multi_names = pd.DataFrame(cond_multi[cond_multi['Source_Name']].index.values)
len(multi_names)

3774

In [52]:
# Taking the components that make up these source names
multi_components = components_cleaned[components_cleaned['Source_Name'].isin(multi_names[0])]
len(multi_components)

9868

In [53]:
multi_components['ID_flag'].value_counts()

31    5407
32    3402
42     768
2      188
41     103
Name: ID_flag, dtype: int64

### Sources with multi components but that were NOT deblended

#### Selecting the sources

In [54]:
# Selecting from Deblended_from empty field
multi_not_deblended = multi_components[multi_components['Deblended_from'] == '']

In [55]:
# Number of components
len(multi_not_deblended)

9007

In [56]:
multi_not_deblended['ID_flag'].value_counts()

31    5407
32    3402
2      188
42      10
Name: ID_flag, dtype: int64

In [57]:
# Which is equal to the number of components = number of pybdsf names (1-1 relation)
len(multi_not_deblended.groupby('Component_Name'))

9007

In [58]:
# This corresponds to the number of sources
len(multi_not_deblended.groupby('Source_Name'))

3565

#### Creating a multi-component sources table

In [59]:
# Multi-component sources table just for the ones that were not deblended
pybdsf_multi_df = pd.DataFrame({
    'pybdsf_name':multi_not_deblended['Component_Name'],
    'flag': 8,
    'association_name': multi_not_deblended['Source_Name']})

In [60]:
pybdsf_multi_df['association_name'].describe()

count                       9007
unique                      3565
top       ILTJ140313.38+542018.9
freq                          35
Name: association_name, dtype: object

### Updating the output table

In [61]:
output_df = output_df.append(pybdsf_multi_df,ignore_index=True)
unittest.main(argv=['ignored', '-v'], exit=False)

test_len_output_df (__main__.Test_output) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


***
## Single sources
***

Single sources are PyBDSF sources that have not been deblended, grouped with other PyBDSFs and are not artifacts. They have a unique correspondence between the PyBDFS raw catalogue and the final optical catalogue. 

### Selecting all the sources that have a unique PyBDSF - Optical source name correspondence

These sources can be selected using the condition defined for multiple sources, using the number of times sources (`Source_Name`) appear on the **components catalogue**.

In [62]:
# Condition to select the sources that are NOT made up of multi components
cond_single = ~cond_multi

In [63]:
# taking the names of these sources
single_names = pd.DataFrame(cond_single[cond_single['Source_Name']].index.values)
len(single_names)

314746

### Selecting the ones that did not went through deblending 

Note that some of these went through a deblending process (i.e. after deblending, one of the sources got the original PyBDSF name and other got a different name).

In [64]:
# Taking the components name for the sources that did not went through deblending
single_components = components_cleaned[(components_cleaned['Source_Name'].isin(single_names[0])) &\
                           (components_cleaned['Deblended_from'] == '')]
len(single_components)

313161

In [65]:
# Confirming
len(single_components.groupby('Component_Name')), len(single_components.groupby('Source_Name'))

(313161, 313161)

### Creating a single sources table

In [66]:
pybdsf_singles_df = pd.DataFrame({
    'pybdsf_name':single_components['Component_Name'],
    'flag':1,
    'association_name': single_components['Source_Name']})

In [67]:
pybdsf_singles_df.head()

,association_name,flag,pybdsf_name
3,ILTJ104327.95+521032.6,1,ILTJ104327.95+521032.6
4,ILTJ104330.98+521038.2,1,ILTJ104330.98+521038.2
5,ILTJ104330.98+515535.8,1,ILTJ104330.98+515535.8
6,ILTJ104332.16+520718.7,1,ILTJ104332.16+520718.7
7,ILTJ104332.19+522351.5,1,ILTJ104332.19+522351.5


### Updating the output table

In [68]:
output_df = output_df.append(pybdsf_singles_df,ignore_index=True) 
unittest.main(argv=['ignored', '-v'], exit=False)

test_len_output_df (__main__.Test_output) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.010s

OK


***
## Artifacts
***

### Artifacts from the (cleaned) artifact catalogue

In [69]:
pybdsf_artifacts = pybdsf_cleaned[pybdsf_cleaned['Source_Name'].
                    isin(artifacts_cleaned['Source_Name'])]
len(pybdsf_artifacts)

2543

Creating a diagnosis artifact table

In [70]:
pybdsf_artifacts_df = pd.DataFrame({
    'pybdsf_name':pybdsf_artifacts['Source_Name'],
    'flag':16,
    'association_name': 'NULL'})

Updating the output table

In [71]:
output_df = output_df.append(pybdsf_artifacts_df, ignore_index=True)
unittest.main(argv=['ignored', '-v'], exit=False)

test_len_output_df (__main__.Test_output) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Sources missing from the optical catalogue are now being classified as artifacts

PyBDSF raw catalogue sources that are not on the output table and not anywhere are now being classified as artifacts

In [72]:
artifacts_not_anywhere = pybdsf_cleaned[~pybdsf_cleaned['Source_Name'].isin(output_df['pybdsf_name'])]
print len(artifacts_not_anywhere)

48


In [73]:
artifacts_not_anywhere_df = pd.DataFrame({
    'pybdsf_name': artifacts_not_anywhere['Source_Name'],
    'flag': 32,
    'association_name': 'NULL'})

Appending the new artifacts the pybdsf artifact table

In [74]:
pybdsf_artifacts_df = pybdsf_artifacts_df.append(artifacts_not_anywhere_df,ignore_index=True)
len(pybdsf_artifacts_df)

2591

### Updating the output table

In [75]:
output_df = output_df.append(artifacts_not_anywhere_df,ignore_index=True)
unittest.main(argv=['ignored', '-v'], exit=False)

test_len_output_df (__main__.Test_output) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


***
## Output table
***

In [76]:
print len(output_df) 
print output_df.dtypes
print output_df['flag'].value_counts()

326591
association_name    object
flag                object
pybdsf_name         object
dtype: object
1.0     313161
8.0       9007
16.0      2543
4.0       1575
12.0       247
32.0        48
3.0         10
Name: flag, dtype: int64


** Creating an astropy table **

In [77]:
output_df['flag'] = pd.to_numeric(output_df['flag'])

In [78]:
output_table = Table.from_pandas(output_df)

** Writing to a fits file **

In [79]:
output_table.write('output_table.fits', overwrite = True)

In [80]:
# Can also be saved in as a csv file directly from the pandas table
# output_df.to_csv('output_table.csv', index=False)